Validation using in silico simulation can be applied as a stand alone procedure for any assignment. The assignments do not need to be generated by MuSiCal.

In [1]:
import musical
import pickle
import numpy as np
import pandas as pd

Read the input data matrix X and the existing assignment table. Here we are using PCAWG assignment as an example. You can replace the data and assignment to do the same for another dataset and signature assignment.

In [ ]:
X = pd.read_csv('data/pcawg/X_CNS.GBM.csv')
H_s = pd.read_csv('data/pcawg/H_CNS.GBM_pcawg.csv')
W_s = pd.read_csv('data/pcawg/W_CNS.GBM_pcawg.csv')

Define a DenovoSig object to run discovery on the dataset and run de novo discovery

In [ ]:
min_n_components = 2
max_n_components = 5
model = musical.DenovoSig(X, 
                          min_n_components = min_n_components, 
                          max_n_components = max_n_components, 
                          init='nndsvdar', 
                          method='mvnmf', 
                          n_replicates=20, 
                          ncpu=10, 
                          mvnmf_lambda_tilde_grid=np.array([1e-10, 1e-9, 5e-9, 1e-8, 5e-8, 1e-7, 2e-7, 5e-7, 1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1]),mv\
                          nmf_hyperparameter_method='single',
                          conv_test_freq=1000,
                          conv_test_baseline='min-iter',
                          max_iter=100000,
                          bootstrap=True,
                          tol=1e-8, 
                          verbose = 1, 
                          normalize_X = True)
model.fit()

It is better to run the lines above on a cluster. You can either wait for the calculation to finish or download the result from the pickle file which contains the result of the above calculation stored in the data folder:

In [ ]:
with open('data/pcawg/musical_output_X_CNS.GBM.pkl', 'rb') as f:
    model = pickle.load(f)

Add the assignments externally and fix the parameters to match the de novo calculation in data. The assignments will be used to generate simulations and run de novo discovery.

In [ ]:
model.n_grid = 1  # there is no grid search for matching and refitting instead we are using an existing assignment                                                                                                                                                
model.H_s = H_s                                                                                                                                                          
model.W_s = W_s           
model.mvnmf_hyperparameter_method = 'fixed'                                                                                                                              
model.mvnmf_lambda_tilde_grid = np.float(model.lambda_tilde_all[model.n_components][0])                                                                                  
model.validate_assignment()                                                                                                                                              

As before you can find the saved result from the above calculation below:

In [ ]:
with open('data/pcawg/musical_output_X_CNS.GBM_validate.pkl', 'rb') as f:
    model = pickle.load(f)

The total distance between de novo signatures from data and simulations:

In [ ]:
print(model.dist_sum_simul)

The distance between de novo signatures from data and simulations for the signature with the largest discrepency

In [ ]:
print(model.dist_max_simul)

plot de novo signatures in data

In [ ]:
model.sigplot_bar(model.W)

plot de novo signatures in simulations

In [ ]:
model.sigplot_bar(model.W_simul)

frobenius norm comparing de novo exposure matrices from data and simulations

In [ ]:
print(model.error_H_simul)